<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/TP1/TP1_M2_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP1

Repositorio de codigo: [GitHub](https://github.com/AgustinBatistelli/programacion_concurrente)

### Implementación usando C++


Utilizo el comando writefile para generar un archivo llamado tp1.cpp. Necesitaré ese archivo para crear un ejecutable. 

In [ ]:
%%writefile tp1.cpp
#include <iostream>
#include <string>
#include <sys/wait.h>
#include <unistd.h>

#define HIJOS_DE_A 3
#define HIJOS_DE_B 2
#define HIJOS_DE_C 0
#define HIJOS_DE_D 1
#define HIJOS_DE_E HIJOS_DE_C
#define HIJOS_DE_F HIJOS_DE_E
#define HIJOS_DE_G HIJOS_DE_F

#define PROCESO_B 0
#define PROCESO_C 1
#define PROCESO_E 0

#define TIEMPO_DORMIDO 30
#define INICIO_ITERADOR 0
#define PROCESO_CREADO_CORRECTAMENTE 0
#define PROCESO_SIN_HIJOS 0
#define SALIDA_EXITOSA EXIT_SUCCESS
#define SALIDA_ERRONEA EXIT_FAILURE

bool procesoCreadoCorrectamente(pid_t pid);
void esperarPorMisHijos(int numeroHijos);
void mostrarMiInformacion(char letra, pid_t miPid, pid_t miParentPid);

bool procesoCreadoCorrectamente(pid_t pid)
{
  return pid >= PROCESO_CREADO_CORRECTAMENTE;
}

void esperarPorMisHijos(int numeroHijos)
{
  for (int i = INICIO_ITERADOR; i < numeroHijos; i++)
  {
    wait(NULL);
  }
}

void mostrarMiInformacion(char letra, pid_t miPid, pid_t miParentPid)
{
  std::cout<<"Soy el proceso "<<letra<<". Mi PID es "<<miPid<<" y el PID de mi padre es "<<miParentPid<<std::endl;
}

//Clase IProceso (simula una Interfaz)
class IProceso 
{
  protected:
    pid_t _miPid;
    pid_t _miPPid;
    char _letra;
    int _cantidadHijos;

  public:
    IProceso(pid_t pid, pid_t ppid, char caracter, int hijos) : _miPid(pid), _miPPid(ppid), _letra(caracter), _cantidadHijos(hijos) 
    {
    }
    virtual void realizarAccion() = SALIDA_EXITOSA;
};

//Procesos que tienen hijos y deben esperarlos
class ProcesoConHijos : public IProceso 
{
  public:
    ProcesoConHijos(pid_t pid, pid_t ppid, char caracter, int hijos) : IProceso(pid, ppid, caracter, hijos) 
    {
    }
    void realizarAccion() override 
    {
      mostrarMiInformacion(this->_letra,this->_miPid,this->_miPPid);
      esperarPorMisHijos(this->_cantidadHijos);
    }
};

//Procesos que no tienen hijos y deben hacer sleep
class ProcesoSinHijos : public IProceso 
{
  public:
    ProcesoSinHijos(pid_t pid, pid_t ppid, char caracter, int hijos) : IProceso(pid, ppid, caracter, hijos) 
    {
    }
    void realizarAccion() override 
    {
      mostrarMiInformacion(this->_letra,this->_miPid,this->_miPPid);
      sleep(TIEMPO_DORMIDO);
    }
};

class ProcesoFabrica
{
  public:
    IProceso* determinarTipoProceso(pid_t pid, pid_t ppid, char caracter, int hijos)
    {
      if (hijos > PROCESO_SIN_HIJOS)
      {
        proceso = new ProcesoConHijos(pid,ppid,caracter,hijos);
      }
      else
      {
        proceso = new ProcesoSinHijos(pid,ppid,caracter,hijos);
      }
      return proceso;
    }
  private:
    IProceso* proceso;
};

int main(int argc, char *argv[])
{
  pid_t pidProcesoA, pidProcesoB, pidProcesoD;
  ProcesoFabrica fabricaDeProcesos;

  for (int i = INICIO_ITERADOR; i < HIJOS_DE_A; i++)
  {
    pidProcesoA = fork();
    if (!procesoCreadoCorrectamente(pidProcesoA))
    {
      std::cout<<"Se ha producido un error al crear un nuevo proceso"<<std::endl;
      return SALIDA_ERRONEA;
    }
    if (!pidProcesoA)
    {
      if (i == PROCESO_B)
      {                
        for (int j = INICIO_ITERADOR; j < HIJOS_DE_B; j++)
        {
          pidProcesoB = fork();
          if (!procesoCreadoCorrectamente(pidProcesoB))
          {
            std::cout<<"Se ha producido un error al crear un nuevo proceso"<<std::endl;
            return SALIDA_ERRONEA;
          }
          if (!pidProcesoB)
          {
            if (j == PROCESO_E)
            {
              IProceso* procesoE = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'E',HIJOS_DE_E);
              procesoE->realizarAccion();
              return SALIDA_EXITOSA;
            }
            else
            {
              IProceso* procesoF = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'F',HIJOS_DE_F);
              procesoF->realizarAccion();
              return SALIDA_EXITOSA;
            }
          }                
        }
        IProceso* procesoB = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'B',HIJOS_DE_B);
        procesoB->realizarAccion();
        return SALIDA_EXITOSA;
      }
      else
      {
        if (i == PROCESO_C)
        {
          IProceso* procesoC = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'C',HIJOS_DE_C);
          procesoC->realizarAccion();
          return SALIDA_EXITOSA;
        }
        else
        {
          pidProcesoD = fork();
          if (!procesoCreadoCorrectamente(pidProcesoD))
          {
            std::cout<<"Se ha producido un error al crear un nuevo proceso"<<std::endl;
            return SALIDA_ERRONEA;
          }
          if (!pidProcesoD)
          {
            IProceso* procesoG = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'G',HIJOS_DE_G);
            procesoG->realizarAccion();
            return SALIDA_EXITOSA;
          }
          IProceso* procesoD = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'D',HIJOS_DE_D);
          procesoD->realizarAccion();
          return SALIDA_EXITOSA;
        }
      }    
    }
  }
  IProceso* procesoA = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'A',HIJOS_DE_A);
  procesoA->realizarAccion();
  return SALIDA_EXITOSA;
}

Overwriting tp1.cpp


Listo los archivos para ver que el archivo tp1.cpp esté creado correctamente.

In [ ]:
!ls -la

total 48
drwxr-xr-x 1 root root  4096 Apr 19 02:22 .
drwxr-xr-x 1 root root  4096 Apr 19 02:20 ..
drwxr-xr-x 4 root root  4096 Apr 14 13:34 .config
-rw-r--r-- 1 root root   418 Apr 19 02:22 salidaC
drwxr-xr-x 1 root root  4096 Apr 14 13:35 sample_data
-rwxr-xr-x 1 root root 19112 Apr 19 02:22 tp1
-rw-r--r-- 1 root root  5203 Apr 19 02:42 tp1.cpp


Compilo el archivo tp1.cpp para obtener un archivo a ejecutar. Este archivo se llamará tp1 y no llevará extensión. 

In [ ]:
!g++ -std=c++11 -o tp1 tp1.cpp

Listo los archivos nuevamente para verificar que tengo el ejecutable del proceso. 

In [ ]:
!ls -la

total 48
drwxr-xr-x 1 root root  4096 Apr 19 02:42 .
drwxr-xr-x 1 root root  4096 Apr 19 02:20 ..
drwxr-xr-x 4 root root  4096 Apr 14 13:34 .config
-rw-r--r-- 1 root root   418 Apr 19 02:22 salidaC
drwxr-xr-x 1 root root  4096 Apr 14 13:35 sample_data
-rwxr-xr-x 1 root root 19112 Apr 19 02:42 tp1
-rw-r--r-- 1 root root  5203 Apr 19 02:42 tp1.cpp


Utilizo el comando nohup para ejecutar el proceso en segundo plano, de manera que no me ocupe la consola. Es necesario añadir el caracter '&' al final.

Para ejecutar el proceso, tengo que poner ./tp1.

El comando 1>salidaC va a redirigir la salida estandar de la pantalla (stdout) a un archivo llamado salidaC.

El comando 2>/dev/null envía la salida de error estandar (stderr) a dev/null

In [ ]:
!nohup ./tp1 1>salidaC 2>/dev/null & 

Al ejecutar el proceso, podré ver al proceso padre y a sus procesos hijos con el comando ps. 

In [ ]:
!ps -ef | grep tp1

root        5812       1  0 02:42 ?        00:00:00 ./tp1
root        5813    5812  0 02:42 ?        00:00:00 ./tp1
root        5814    5812  0 02:42 ?        00:00:00 ./tp1
root        5815    5812  0 02:42 ?        00:00:00 ./tp1
root        5816    5813  0 02:42 ?        00:00:00 ./tp1
root        5817    5813  0 02:42 ?        00:00:00 ./tp1
root        5818    5815  0 02:42 ?        00:00:00 ./tp1
root        5831     518  0 02:42 ?        00:00:00 /bin/bash -c ps -ef | grep tp1
root        5833    5831  0 02:42 ?        00:00:00 grep tp1


Utilizo el comando pstree para poder obtener el árbol de procesos. Si le paso el PID del proceso padre, podré ver tanto su PID como el PID de sus hijos y nietos. 

In [ ]:
!pstree -p 5812

tp1(5812)─┬─tp1(5813)─┬─tp1(5816)
          │           └─tp1(5817)
          ├─tp1(5814)
          └─tp1(5815)───tp1(5818)


Utilizo el comando cat para poder mostrar el contenido del archivo salidaC, que es a donde se fueron redirigidos los print que realizan cada uno de los procesos. 

In [ ]:
!cat salidaC

Soy el proceso A. Mi PID es 5812 y el PID de mi padre es 1
Soy el proceso B. Mi PID es 5813 y el PID de mi padre es 5812
Soy el proceso F. Mi PID es 5817 y el PID de mi padre es 5813
Soy el proceso C. Mi PID es 5814 y el PID de mi padre es 5812
Soy el proceso E. Mi PID es 5816 y el PID de mi padre es 5813
Soy el proceso D. Mi PID es 5815 y el PID de mi padre es 5812
Soy el proceso G. Mi PID es 5818 y el PID de mi padre es 5815


### Implementacion usando Python

Escribo el archivo fork.py con la logica del programa

In [ ]:
%%writefile fork.py
import os
import time

def son(data, value = None):
 if value == None:
  value = data.get('start')
 
 children_pids = []
 if value in data.get('processes').keys(): 
  for son_value in data.get('processes').get(value):
   pid = os.fork()
   if pid:
    children_pids.append(pid)
   else:
    son(data, son_value)
    return
    
  for child_pid in children_pids:
   os.waitpid(child_pid, 0)

 print('My value is: {}. My PID is: {}. My PPID is: {}'.format(value, os.getpid(), os.getppid())) 
 time.sleep(25)

if __name__ == '__main__':
 data = {'start': 'A','processes': {'A': ['B', 'C', 'D'],'B': ['E', 'F'],'D': ['G']}}
 son(data)

Writing fork.py


Ejecutamos el programa en segundo plano

In [ ]:
!nohup python fork.py 1>salidaPython 2>/dev/null &

Buscamos con ps el proceso que esta ejecutando nuestro programa

In [ ]:
!ps -ef | grep fork.py


root         488       1  1 16:44 ?        00:00:00 python3 fork.py
root         490     488  0 16:44 ?        00:00:00 python3 fork.py
root         491     488  0 16:44 ?        00:00:00 python3 fork.py
root         492     488  0 16:44 ?        00:00:00 python3 fork.py
root         493     490  0 16:44 ?        00:00:00 python3 fork.py
root         494     490  0 16:44 ?        00:00:00 python3 fork.py
root         495     492  0 16:44 ?        00:00:00 python3 fork.py
root         508     166  0 16:44 ?        00:00:00 /bin/bash -c ps -ef | grep fork.py
root         510     508  0 16:44 ?        00:00:00 grep fork.py


Mostramos el árbol de procesos que se generó

In [ ]:
!pstree -pc 488               

python3(488)─┬─python3(490)─┬─python3(493)
             │              └─python3(494)
             ├─python3(491)
             └─python3(492)───python3(495)


Mostrar la salida del proceso ejecutandolo nuevamente

In [ ]:
!python fork.py

Proceso  A  con PID:  8070  y PID de padre:  158
Proceso  B  con PID:  8072  y PID de padre:  8070
Proceso  D  con PID:  8074  y PID de padre:  8070
Proceso  F  con PID:  8076  y PID de padre:  8072
Proceso  C  con PID:  8073  y PID de padre:  8070
Proceso  E  con PID:  8075  y PID de padre:  8072
Proceso  G  con PID:  8077  y PID de padre:  8074


### Implementacion usando JAVA

Descargar la libreria gson para java.

In [ ]:
!curl https://repo1.maven.org/maven2/com/google/code/gson/gson/2.10.1/gson-2.10.1.jar --output "gson-2.10.1.jar"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  276k  100  276k    0     0  1976k      0 --:--:-- --:--:-- --:--:-- 1976k


Validar la descarga de la libreria

In [ ]:
!ls -lash

total 332K
4.0K drwxr-xr-x 1 root root 4.0K Apr 12 16:44 .
4.0K drwxr-xr-x 1 root root 4.0K Apr 12 16:42 ..
4.0K drwxr-xr-x 4 root root 4.0K Apr 11 13:32 .config
4.0K -rw-r--r-- 1 root root 2.1K Apr 12 16:44 fork.py
280K -rw-r--r-- 1 root root 277K Apr 12 16:44 gson-2.10.1.jar
4.0K -rw-r--r-- 1 root root  418 Apr 12 16:43 salidaC
   0 -rw-r--r-- 1 root root    0 Apr 12 16:44 salidaPython
4.0K drwxr-xr-x 1 root root 4.0K Apr 11 13:33 sample_data
 20K -rwxr-xr-x 1 root root  19K Apr 12 16:43 tp1
8.0K -rw-r--r-- 1 root root 5.1K Apr 12 16:43 tp1.cpp


Generar el archivo con datos de entrada.

In [ ]:
!echo '{"A":["B","C","D"],"B":["E","F"],"D":["G"]}' > data.json

Generar el archivo con el programa necesario.

In [ ]:
%%writefile ProcessStuff.java
import com.google.gson.Gson;
import com.google.gson.reflect.TypeToken;
import java.lang.reflect.Type;
import java.io.StringReader;
import java.io.PrintWriter;
import java.io.FileOutputStream;
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.FileSystems;
import java.nio.file.FileSystem;

import java.util.*;

public class ProcessTree 
{
  public static void main(String[] args) 
  {
    String self = (args.length == 0) ? "A" : String.valueOf(args[0]);
    
    info(self);
    waitForChildren(spawnChildren(getDataMap(), self));

    try 
    {
      Thread.sleep(5000);
    } catch (InterruptedException e) 
    {
      System.err.println("Se ha producido una interrupción");
    }

  }

  public static void waitForChildren(List children) 
  {
    for(Process child : children) 
    {
      try 
      {
        child.waitFor();
      } catch (InterruptedException e) 
      {
        System.err.println("Se ha producido una interrupción");
      }
    }
  }

  public static HashMap> getDataMap() 
  {
    Gson gson = new Gson();
    Type customType = new TypeToken>>() {}.getType();
    return gson.fromJson(getFileData(DATA_FILE_NAME), customType); 
  }

  public static String getFileData(String fileName) 
  {
    try 
    {
      return String.join("", 
        Files.readAllLines(FileSystems.getDefault().getPath(fileName))
      );
    } catch (NullPointerException e) 
    {
      e.printStackTrace();
    } catch (IOException e) 
    {
      System.err.println("No se pudo leer el archivo " + fileName);
      e.printStackTrace();
    }
    return "";
  }

  public synchronized static void info(String selfData) 
  {
    try ( PrintWriter out = new PrintWriter(new FileOutputStream(OUT, true), true); ) 
    {
      out.print("Soy el proceso " + selfData + ". ");
      out.print("Mi PID es: " + ProcessHandle.current().pid() + ". ");
      out.print("El PID de mi padre es: " + ProcessHandle.current().parent().get().pid() + ".\n");
      out.flush();
    } catch (IOException e) 
    {
      System.err.println("No se pudo abrir el archivo: " + OUT);
      e.printStackTrace();
    } catch (NoSuchElementException e) 
    {
      System.err.println("No tengo padre: " + OUT);
      e.printStackTrace();
    }
  }

  public static ArrayList spawnChildren(HashMap> dataMap, String self) 
  {
    ArrayList children = new ArrayList<>();
    if (dataMap.keySet().contains(self)) 
    {
      for(String childValue : dataMap.get(self)) 
      {
        try 
        {
          children.add(new ProcessBuilder("java", "-cp", ".:gson-2.10.1.jar:gson-2.10.1.jar:", "ProcessTree", childValue, "1>salidaJava", "2>errors").start());
        } catch (IOException e) 
        {
          System.err.println("No se pudieron crear mas procesos:  " + childValue);
        }
      }
    }
    return children;
  }

  final static String DATA_FILE_NAME = "target.json";
  final static String OUT = "miSalidaJava";
}

Overwriting ProcessStuff.java


Compilar el programa junto con la libreria gson.

In [ ]:
!javac -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff.java

Ejecutar el programa en segundo plano.

In [ ]:
!nohup java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff 1>salidaJava 2>errors &  

In [ ]:
!ps -ef | grep java

root       13364       1 12 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff
root       13381   13364 14 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff B 1>salidaJava 2>errors
root       13384   13364 14 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff C 1>salidaJava 2>errors
root       13387   13364 14 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff D 1>salidaJava 2>errors
root       13439   13387 23 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff G 1>salidaJava 2>errors
root       13444   13381 23 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff E 1>salidaJava 2>errors
root       13453   13381 21 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff F 1>salidaJava 2>errors
root       13501     166  0 17:32 ?        00:00:00 /bin/bash -c ps -ef | grep java
root     

Mostrar el arbol de procesos generado

In [ ]:
!pstree -pcT "$(eval ps -eo pid,ppid,args | awk '$NF ~ /ProcessStuff/ && $2 == 1 {print $1}')"                              

java(13364)─┬─java(13381)─┬─java(13444)
            │             └─java(13453)
            ├─java(13384)
            └─java(13387)───java(13439)


Mostrar resultado del archivo de salida del proceso

In [ ]:
!cat miSalidaJava

Soy el proceso A. Mi PID es: 13364. El PID de mi padre es: 1.
Soy el proceso D. Mi PID es: 13387. El PID de mi padre es: 13364.
Soy el proceso C. Mi PID es: 13384. El PID de mi padre es: 13364.
Soy el proceso B. Mi PID es: 13381. El PID de mi padre es: 13364.
Soy el proceso E. Mi PID es: 13444. El PID de mi padre es: 13381.
Soy el proceso G. Mi PID es: 13439. El PID de mi padre es: 13387.
Soy el proceso F. Mi PID es: 13453. El PID de mi padre es: 13381.


Borrar el archivo, en caso de volver a ejecutar.

In [ ]:
!rm miSalidaJava